<a href="https://colab.research.google.com/github/hlmth/projet-python/blob/main/ConcatBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.3 MB/s eta 0:00:00


In [3]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import os
import copy
import pandas as pd
from transformers import AutoTokenizer, AutoModel, BertModel
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)


PyTorch Version:  2.0.0+cu118
Torchvision Version:  0.15.1+cu118
cuda:0


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
num_epochs = 1

batch_size = 8

max_len = 336
# Chargement du tokenizer et du modèle BERT
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)



In [5]:
train = pd.read_json("/content/drive/MyDrive/data/train.jsonl",lines=True)
test = pd.read_json("/content/drive/MyDrive/data/val.jsonl",lines=True)
del train['id']
del test['id']
def stop_words(df, column, new_column):
  df[new_column]=df[column].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords.words('english')]))
  return df

def punctuation(df, column, new_column):
  df[new_column]=df[column].apply(lambda x: "".join([char for char in x if char not in string.punctuation]))
  return df

In [6]:
cleaned_train = stop_words(train, 'text', 'cleaned_text')
cleaned_train = punctuation(cleaned_train, 'cleaned_text', 'cleaned_text')
del cleaned_train['text']

In [7]:
cleaned_test = stop_words(test, 'text', 'cleaned_text')
cleaned_test = punctuation(cleaned_test, 'cleaned_text', 'cleaned_text')
del cleaned_test['text']

In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(cleaned_train, test_size=1000, random_state=42)

# Verify the sizes of the datasets
print(f'Train dataset size: {len(train_df)}')
print(f'Validation dataset size: {len(val_df)}')

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


Train dataset size: 7500
Validation dataset size: 1000


In [9]:
import torch
from PIL import Image

class MyDataset(torch.utils.data.Dataset):
  'Caractérise un jeu de données pour PyTorch'
  def __init__(self, transforms = None, root_dir = '/content/drive/MyDrive/data', mode = 'train', tokenizer = tokenizer):
        'Initialisation'
        if mode == 'train' : 
            self.df = train_df
        elif mode == 'val' :
            self.df = val_df
        elif mode == 'test':
            self.df = cleaned_test
        self.labels = self.df.label
        self.image_names = self.df.img
        self.transforms = transforms
        self.root_dir = root_dir
        self.texts = [tokenizer(sentence, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt") for sentence in self.df['cleaned_text']]

#        print(len(self.labels[self.labels == 0])/len(self.labels))
      
  def __len__(self):
        "Représente le nombre total d'exemples du jeu de données"
        return len(self.labels)

  def __getitem__(self, idx):


      image_path = f"{self.root_dir}/{self.image_names.iloc[idx]}"

      img = Image.open(image_path, ).convert('RGB')

      if self.transforms :
            img = self.transforms(img)

      

      return self.texts[idx], img, self.labels.iloc[idx]
  


In [10]:
# Data augmentation and normalization for training
# Just normalization for validation
input_size = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomResizedCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        
    ]),
    'test': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomResizedCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),

    ])
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: MyDataset(transforms = data_transforms[x], mode = x) for x in ['train', 'val', 'test']}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:
image_datasets

{'train': <__main__.MyDataset at 0x7f4450164670>,
 'val': <__main__.MyDataset at 0x7f44501649a0>,
 'test': <__main__.MyDataset at 0x7f4450164a30>}

In [12]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    else :
        for param in model.parameters():
            param.requires_grad = True

def initialize_model( feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    model_ft = models.resnet18(pretrained=use_pretrained)
    
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 1)
    set_parameter_requires_grad(model_ft, feature_extract)
    return model_ft

In [13]:
from torch import nn
from transformers import BertForSequenceClassification
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input_id, mask):

        bert_output = self.bert(input_ids = input_id, attention_mask = mask)
        logits = bert_output.logits          # pour obtenir le tensor en sortie de bert
        final_output = self.dropout(logits)  # pour réduire l'overfitt, ici p = 0.5

        return logits

In [14]:
# définir le modèle Bert
bert_model = BertClassifier()
bert_model.load_state_dict(state_dict = torch.load('/content/drive/MyDrive/data/best_model_sigm (1).pth', map_location=torch.device('cpu')))

# définir le modèle ResNet18
resnet18_model = initialize_model( feature_extract= True )
resnet18_model.load_state_dict(torch.load('/content/drive/MyDrive/data/best_model.pth'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [15]:
bert_model

BertClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [16]:
# définir une classe de modèle qui effectue la moyenne des sorties des deux modèles
class ConcatBertModel(torch.nn.Module):
    def __init__(self, bert_model, resnet18_model):
        super(ConcatBertModel, self).__init__()
        self.bert_model = bert_model
        self.resnet18_model = resnet18_model
        self.fc = torch.nn.Linear(3, 1)
        set_parameter_requires_grad(self.bert_model ,True)
        set_parameter_requires_grad(self.resnet18_model ,True)
        
    def forward(self, input_ids, attention_mask, image):
        bert_output = self.bert_model(input_id=input_ids, mask=attention_mask)

        resnet_output = self.resnet18_model(image)
      
        concat_output = torch.cat((bert_output, resnet_output), dim=1)
        output = self.fc(concat_output)

        output = torch.sigmoid(output)
        return output


In [24]:
# initialiser une instance de ConcatBertModel
concat_model = ConcatBertModel(bert_model, resnet18_model)
concat_model = concat_model.to(device)
print(concat_bert)

ConcatBertModel(
  (bert_model): BertClassifier(
    (bert): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOu

In [18]:
def train_concat_bert(model, dataloaders, criterion, optimizer, num_epochs):
    since = time.time()
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.58

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_TP = 0.0
            running_FP = 0.0
            running_TN = 0.0
            running_FN = 0.0
            for input_ids, inputs, labels in tqdm(dataloaders[phase]):
                TP=FP=TN=FN=0
                inputs = inputs.to(device)
                input_id = input_ids['input_ids'].squeeze(1).to(device)
                mask = input_ids['attention_mask'].to(device)

                labels = labels.to(device)
              
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # effectuer la prédiction et calculer la perte
                    outputs = concat_bert(input_id, mask, inputs)
                    loss = criterion(outputs, labels.float().unsqueeze(1).to(device))
                
                    if phase == 'train':
                        # rétropropager et mettre à jour les poids
                        loss.backward()
                        optimizer.step()

                  
                    preds = (outputs > 0.5).float()
                    for i in range(len(preds)) :
                      if preds[i]== 1 and labels[i]==1 : TP+=1
                      elif preds[i]== 1 and labels[i]==0 : FP+=1
                      elif preds[i]== 0 and labels[i]==0 : TN+=1
                      elif preds[i]== 0 and labels[i]==1 : FN+=1
                    

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.float().unsqueeze(1))
                running_TP += TP
                running_FP += FP
                running_TN += TN
                running_FN += FN
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_recall = running_TP / (running_TP + running_FN) # recall
            epoch_specificity = running_TN / (running_TN + running_FP)
            print('{} Loss: {:.4f} Acc: {:.4f} Recall: {:.4f} Specificity: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_recall, epoch_specificity))
            
                # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), "/content/drive/MyDrive/data/best_model_concat.pth")

            if phase == 'val':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return concat_bert, val_acc_history

In [19]:

def test(model, criterion, device, batch_size):
    model.eval()
    model.to(device)
    test_loader = dataloaders_dict['test']
    running_loss = 0.0
    running_corrects = 0
    running_TP = 0.0
    running_FP = 0.0
    running_TN = 0.0
    running_FN = 0.0
    with torch.no_grad():
        for input_ids, inputs, labels in tqdm(test_loader):
            TP=FP=TN=FN=0
            inputs = inputs.to(device)
            input_id = input_ids['input_ids'].squeeze(1).to(device)
            mask = input_ids['attention_mask'].to(device)
            labels = labels.to(device)
         
            outputs = concat_bert(input_id, mask, inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1).to(device))
            preds = (outputs > 0.5).float()
            for i in range(len(preds)) :
              if preds[i]== 1 and labels[i]==1 : TP+=1
              elif preds[i]== 1 and labels[i]==0 : FP+=1
              elif preds[i]== 0 and labels[i]==0 : TN+=1
              elif preds[i]== 0 and labels[i]==1 : FN+=1
                    

                # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.float().unsqueeze(1))
            running_TP += TP
            running_FP += FP
            running_TN += TN
            running_FN += FN
    test_loss = running_loss / len(test_loader.dataset)
    test_acc = running_corrects.double() / len(test_loader.dataset)
    test_recall = running_TP / (running_TP + running_FN) # recall
    test_specificity = running_TN / (running_TN + running_FP)
    print('{} Loss: {:.4f} Acc: {:.4f} Recall: {:.4f} Specificity: {:.4f}'.format('test', test_loss, test_acc, test_recall, test_specificity))
            


In [20]:
# définir la fonction de perte et l'optimiseur
#criterion = torch.nn.BCEWithLogitsLoss(weight=torch.tensor([1, 1.787]))
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(concat_bert.parameters(), lr=1e-4)

In [21]:
print("Params to learn:")
params_to_update = []
for name,param in concat_bert.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [32]:
concat_model, hist = train_concat_bert(concat_model, dataloaders_dict, criterion, optimizer, num_epochs)

Epoch 0/0
----------


100%|██████████| 938/938 [03:21<00:00,  4.66it/s]


train Loss: 0.6670 Acc: 0.8100 Recall: 0.5902 Specificity: 0.9337


100%|██████████| 125/125 [00:28<00:00,  4.43it/s]


val Loss: 0.6520 Acc: 0.8640 Recall: 0.6819 Specificity: 0.9616
Training complete in 3m 52s
Best val Acc: 0.864000


In [33]:
test(concat_model, criterion, device, batch_size)

100%|██████████| 63/63 [00:16<00:00,  3.77it/s]

test Loss: 0.6861 Acc: 0.5540 Recall: 0.2000 Specificity: 0.9080


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score



# Prédire les probabilités pour les données de test
concat_model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for input_ids, inputs, labels in dataloaders_dict['test']:
        inputs = inputs.to(device)
        input_id = input_ids['input_ids'].squeeze(1).to(device)
        mask = input_ids['attention_mask'].to(device)

        labels = labels
              
        outputs = concat_bert(input_id, mask, inputs)

        y_pred.extend(torch.sigmoid(outputs).cpu().numpy())
        y_true.extend(labels.numpy())


# Calculer l'AUC-ROC
auc_roc = roc_auc_score(y_true, y_pred)
print("AUC-ROC : {:.4f}".format(auc_roc))

# Calculer la courbe ROC
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve')
plt.legend(loc="lower right")
plt.show()